In [ ]:
from datetime import datetime, timedelta, date
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [ ]:
sg = Subgrounds()
holders = sg.load_api('https://api.studio.thegraph.com/query/28103/token-holders/0.0.23')

In [12]:
import os


def do_query(holders, page, start_date: datetime, finish_date: datetime, df):
    # Fetch the next page and concatenate
    events_query = holders.Query.tokenHolderTransactions(
        orderBy="timestamp",
        orderDirection="asc",
        first=1000,
        skip=1000*page,
        where=[
            holders.TokenHolderTransaction.date >= start_date.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
            holders.TokenHolderTransaction.date < finish_date.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
        ]
    )
    print("Fetching records for date: ", start_date)
    print("page: ", page)
    events = sg.query_df([events_query.date, events_query.value, events_query.holder.holder, events_query.holder.token.name])
    print("records: ", len(events))
    updated_df = pd.concat([df, events])

    if len(events) < 1000:
        return updated_df

    return do_query(holders, page+1, start_date, finish_date, updated_df)

df = pd.DataFrame()

if os.path.exists("results.csv"):
    print("Reading cached results")
    df = pd.read_csv("results.csv")
else:
    # For each day, fetch the day's events
    start_date = datetime(2021,11,24)
    today = datetime.today()
    delta = timedelta(hours=6)

    while start_date <= today:
        next_date = start_date + delta

        # Fetch recursively
        df = do_query(holders, 0, start_date, next_date, df)

        # Increment start date for the next loop
        start_date += delta

    # Cache the results
    df.to_csv("results.csv", float_format="%.18f")
    print("Wrote cached results")

# Parse the datetime string and convert to date
print("Parsing date")
new_df = df.assign(tokenHolderTransactions_date=pd.to_datetime(df["tokenHolderTransactions_date"], format="%Y-%m-%dT%H:%M:%S.000Z").dt.date)

# Calculate daily balance
print("Calculating daily sum")
grouped = new_df.groupby(["tokenHolderTransactions_holder_holder", "tokenHolderTransactions_holder_token_name", "tokenHolderTransactions_date"]).sum()

# Generate rows for each date/token/holder permutation
print("Generating permutations")
dt = grouped.index.get_level_values("tokenHolderTransactions_date").unique()
idx = pd.MultiIndex.from_product([grouped.index.get_level_values("tokenHolderTransactions_holder_holder").unique(), grouped.index.get_level_values("tokenHolderTransactions_holder_token_name").unique(), pd.date_range(dt.min(), dt.max(), freq="D")], names=["Holder", "Token", "Date"])
reindexed = grouped.reindex(idx).groupby(["Holder", "Token", "Date"], sort=False).fillna(0).reset_index()

# Calculate the cumulative balance
print("Calculating daily cumulative balance")
reindexed["balance"] = reindexed.groupby(["Holder", "Token"]).tokenHolderTransactions_value.cumsum()

# Write to file
print("Writing to records.csv")
reindexed.to_csv("records.csv", index=False, float_format="%.18f")

# Cache balances by day
# Don't calculate zero balances
# Use previous day balances as starting point